In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import RidgeCV

In [2]:
def transform_data(X):
    """
    This function transforms the 5 input features of matrix X (x_i denoting the i-th component of X) 
    into 21 new features phi(X) in the following manner:
    5 linear features: phi_1(X) = x_1, phi_2(X) = x_2, phi_3(X) = x_3, phi_4(X) = x_4, phi_5(X) = x_5
    5 quadratic features: phi_6(X) = x_1^2, phi_7(X) = x_2^2, phi_8(X) = x_3^2, phi_9(X) = x_4^2, phi_10(X) = x_5^2
    5 exponential features: phi_11(X) = exp(x_1), phi_12(X) = exp(x_2), phi_13(X) = exp(x_3), phi_14(X) = exp(x_4), phi_15(X) = exp(x_5)
    5 cosine features: phi_16(X) = cos(x_1), phi_17(X) = cos(x_2), phi_18(X) = cos(x_3), phi_19(X) = cos(x_4), phi_20(X) = cos(x_5)
    1 constant features: phi_21(X)=1

    Parameters
    ----------
    X: matrix of floats, dim = (700,5), inputs with 5 features

    Returns
    ----------
    X_transformed: array of floats: dim = (700,21), transformed input with 21 features
    """
    X_transformed = np.zeros((len(X), 21))
    
    phi_1 = X[:,0]
    phi_2 = X[:,1]
    phi_3 = X[:,2]
    phi_4 = X[:,3]
    phi_5 = X[:,4]
    phi_6 = phi_1**2
    phi_7 = phi_2**2
    phi_8 = phi_3**2
    phi_9 = phi_4**2
    phi_10 = phi_5**2
    phi_11 = np.exp(phi_1)
    phi_12 = np.exp(phi_2)
    phi_13 = np.exp(phi_3)
    phi_14 = np.exp(phi_4)
    phi_15 = np.exp(phi_5)
    phi_16 = np.cos(phi_1)
    phi_17 = np.cos(phi_2)
    phi_18 = np.cos(phi_3)
    phi_19 = np.cos(phi_4)
    phi_20 = np.cos(phi_5)
    phi_21 = np.ones(len(X))
    X_transformed = np.array([phi_1, phi_2, phi_3, phi_4, phi_5, phi_6, phi_7, phi_8, phi_9, phi_10, phi_11, phi_12, phi_13, phi_14, phi_15, phi_16, phi_17, phi_18, phi_19, phi_20, phi_21]).T

    assert X_transformed.shape == (len(X), 21)
    return X_transformed


In [3]:
def calculate_RMSE(w, X, y):
    """This function takes test data points (X and y), and computes the empirical RMSE of 
    predicting y from X using a linear model with weights w. 

    Parameters
    ----------
    w: array of floats: dim = (13,), optimal parameters of ridge regression 
    X: matrix of floats, dim = (15,13), inputs with 13 features
    y: array of floats, dim = (15,), input labels

    Returns
    ----------
    RMSE: float: dim = 1, RMSE value
    """
    X_transformed = transform_data(X)
    RMSE = np.sqrt(np.mean((y - np.dot(X_transformed, w))**2))
    assert np.isscalar(RMSE)
    return RMSE


In [4]:
def fit(X, y, ):
    """
    This function receives training data points, transform them, and then fits the linear regression on this 
    transformed data. Finally, it outputs the weights of the fitted linear regression. 

    Parameters
    ----------
    X: matrix of floats, dim = (700,5), inputs with 5 features
    y: array of floats, dim = (700,), input labels)

    Returns
    ----------
    w: array of floats: dim = (21,), optimal parameters of linear regression
    """
    w = np.zeros((21,))
    X_transformed = transform_data(X)
    # X_abs_max = np.max(np.abs(X_transformed), axis=0)
    # X_transformed = X_transformed / X_abs_max
    alpha_list = np.logspace(-3, 3, 1000)
    ridge = RidgeCV(alphas=alpha_list, fit_intercept=False, cv=10)
    ridge.fit(X_transformed, y)
    print('alpha =', ridge.alpha_)
    w = ridge.coef_
    # w = w / X_abs_max
    assert w.shape == (21,)
    return w

In [5]:
# Data loading
data = pd.read_csv("train.csv")
y = data["y"].to_numpy()
data = data.drop(columns=["Id", "y"])
# print a few data samples
print(data.head())
X = data.to_numpy()
# The function retrieving optimal LR parameters
w = fit(X, y)
# Save results in the required format
RMSE = calculate_RMSE(w, X, y)
print('RMSE is ', RMSE)
np.savetxt("./results-ridge.csv", w, fmt="%.12f")

     x1    x2    x3    x4    x5
0  0.02  0.05 -0.09 -0.43 -0.08
1 -0.13  0.11 -0.08 -0.29 -0.03
2  0.08  0.06 -0.07 -0.41 -0.03
3  0.02 -0.12  0.01 -0.43 -0.02
4 -0.14 -0.12 -0.08 -0.02 -0.08
alpha = 40.4209583979631
RMSE is  1.9541133268845612
